In [1]:
import requests
import lxml
import csv
import re
import pandas as pd
from bs4 import BeautifulSoup
import pyarrow
import time
import random
import pyarrow.feather as feather
from tqdm import *
from lxml import etree

In [2]:
#cookies是分城市的
def get_xiaoqu_chengjiao(xiaoqulinks,cookies):
    user_agenrs = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
        ]
    df_cols = [ '城市',
                '小区名称',
                '小区id',
                '挂牌价格（万）',
                '成交周期（天）',
                '调价（次）',
                '成交日期',
                '浏览（次）',
                '链家编号',
                '所在区域',
                '房屋户型',
                '所在楼层',
                '建筑面积',
                '户型结构',
                '套内面积',
                '建筑类型',
                '房屋朝向',
                '装修情况',
                '建筑结构',
                '梯户比例',
                '配备电梯',
                '交易权属',
                '挂牌时间',
                
                '房屋用途',
                '房屋年限',
                '产权所属',
                '关注次数',
                '看房次数',
                '房屋链接']
    result_house_df = pd.DataFrame()
   
    #count=0
    
    for link in tqdm(xiaoqulinks):
        agent = random.choice(user_agenrs)
        parts=link.split('/')
        #把城市简称提出来
        get_city=parts[2].split('.')
        a_city=get_city[0]
        head0={'User-Agent':agent}
        link_r=requests.get(link,headers=head0)
        link_html=etree.HTML(link_r.text)
        xiaoqu_name_path=link_html.xpath("//div[@class='xiaoquDetailHeaderContent clear']/div[@class='detailHeader fl']/h1[@class='detailTitle']/text()")
        if xiaoqu_name_path:
            xiaoqu_name=xiaoqu_name_path[0].strip()
        else:
            xiaoqu_name="NA"
        
        xiaoqu_diqu_path=link_html.xpath("//div[@class='xiaoquDetailHeaderContent clear']/div[@class='detailHeader fl']/div[@class='detailDesc']/text()")
        if xiaoqu_diqu_path:
            xiaoqu_diqu=xiaoqu_diqu_path[0].strip()
        else:
            xiaoqu_diqu="NA"
        
        url=f'https://{parts[2]}/chengjiao/c{parts[4]}/'
        head={'Host':parts[2],
             'Referer':url,
             'User-Agent':agent,
             'cookie':cookies}
        source_r=requests.get(url,headers=head)
        source_html=etree.HTML(source_r.text)
        
        #count+=1
        if len(source_html.xpath("//ul[@class='listContent']//li//a[@class='img']/@href")) ==0:
            # print(city_name,xiaoqu_info_df['district'][i],xiaoqu_info_df['area'][i],xiaoqu_info_df['name'][i],'|',xiaoqu_info_df['id'][i],'无成交记录')
            pass
        else :
            for page in range (1,101):
                target_url =f'https://{parts[2]}/chengjiao/pg {str(page)}c{parts[4]}/'
                r = requests.get(target_url,headers=head)
                html=etree.HTML(r.text)
                #count+=1
                if len(html.xpath("//ul[@class='listContent']//li//a[@class='img']//@href")) >0 :
                    # print('in')

                    house_url_list = html.xpath("//ul[@class='listContent']//li//a[@class='img']//@href")
                    
                    house_deal_date_list = html.xpath("//div[@class='dealDate']//text()")
                    
                    # print(house_url_list)
                    # print(house_deal_date_list)
                    house_head = {
                        'Host': parts[2],
                        'Referer': target_url,
                        'User-Agent': agent,
                        'cookie':cookies
                        }

                    #指标：城市、地区、小区名称、小区代码、参考总价、参考单价、房屋户型、所在楼层、户型结构、建筑面积、房屋朝向、建筑类型、建筑结构、梯户比例
                    #配备电梯、装修情况、挂牌时间、上次交易、交易权属、房屋用途、产权所属、房屋年限、产权所属、抵押信息、房本备件

                    city_list=[]
                    xiaoqu_name_list=[]
                    
                    xiaoqu_id_list=[]
                    
                    guapai_price_list=[]
                    chengjiao_zhouqi_list=[]
                    house_tiaojia_list=[]
                    house_lianjia_list=[]
                    house_liulan_list=[]
                    xiaoqu_diqu_list=[]
                    house_huxing_list = []
                    house_louceng_list = []
                    house_mianji_list = []
                    house_hxjiegou_list = []
                    house_taonei_list = []
                    house_leixing_list = []
                    house_chaoxiang_list = []
                    '''
                    house_niandai_list = []
                    '''
                    house_zhuangxiu_list = []
                    house_jzjiegou_list = []
                    '''
                    house_gongnuan_list = []
                    '''
                    house_tihu_list = []
                    house_dianti_list = []
                    house_jiaoyiquanshu_list = []
                    house_guapai_list = []
                    
                    house_yongtu_list = []
                    house_nianxian_list = []
                    house_suoshu_list = []
                    '''
                    house_dealTotalPrice_list = []
                    house_avg_price_list = []
                    house_guapai_price_list = []
                    house_chengjiaozhouqi_list = []
                    house_tiaojia_list = []
                    house_daikan_list = []
                    '''
                    house_guanzhu_list = []
                    
                    house_kanguo_list = []
                    for j in range(0,len(house_url_list)):
    
                        house_url = house_url_list[j] 
                        '''
                        house_deal_date = house_deal_date_list[j]
                        '''
                        #print(house_url)
                        house_html = etree.HTML(requests.get(house_url,headers=house_head).text)
                        '''
                        sec = random.randint(1,15)
                        print("sleep",sec)
                        time.sleep(sec)
                        '''
                        #count = count+1
                        # if len((house_html.xpath("//div[@class='row']//h1[contains(text(),'人机认证')]//text()")))>0:
                        # print((house_html.xpath("//div[@class='row']//h1//text()"))[0].replace(' ',''))
                        lianjia_bianhao = (house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'链家编号')]/../text()")[0]).replace(' ','')
                        guapai_price = (house_html.xpath("//div[@class='msg']/span[contains(text(),'挂牌价格（万）')]/label/text()")[0]).replace(' ','')
                        chengjiao_zhouqi=(house_html.xpath("//div[@class='msg']/span[contains(text(),'成交周期（天）')]/label/text()")[0]).replace(' ','')
                        house_tiaojia=(house_html.xpath("//div[@class='msg']/span[contains(text(),'调价（次）')]/label/text()")[0]).replace(' ','')
                        house_liulan=(house_html.xpath("//div[@class='msg']/span[contains(text(),'浏览（次）')]/label/text()")[0]).replace(' ','')
                        house_lianjia=(house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'链家编号')]/../text()")[0]).replace(' ','')
                       
                        
                        huxing = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'房屋户型')]/../text()")
                        if huxing:
                            house_huxing = huxing[0].strip()
                        else:
                            house_huxing = "NA"
                        
                        
        
                        louceng = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'所在楼层')]/../text()")
                        if louceng:
                            house_louceng = louceng[0].strip()
                        else:
                            house_louceng = "NA"
                
                        mianji = house_html.xpath ("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'建筑面积')]/../text()")
                        if mianji:
                            house_mianji = mianji[0].strip()
                        else:
                            house_mianji = "NA"
                        
                        
                        hxjiegou = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'户型结构')]/../text()")
                        if hxjiegou:
                            house_hxjiegou = hxjiegou[0].strip()
                        else:
                            house_hxjiegou = "NA"
                        
                        
                        taonei = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'套内面积')]/../text()")
                        if taonei:
                            house_taonei = taonei[0].strip()
                        else:
                            house_taonei = "NA"
                        
                        
                        leixing = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'建筑类型')]/../text()")
                        if leixing:
                            house_leixing = leixing[0].strip()
                        else:
                            house_leixing = "NA"
                        
                        
                        chaoxiang = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'房屋朝向')]/../text()")
                        if chaoxiang:
                            house_chaoxiang = chaoxiang[0].strip()
                        else:
                            house_chaoxiang = "NA"
                        
                        
                        '''
                        house_niandai = (house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'建成年代')]/../text()")[0]).replace(' ','')
                        '''
                        zhuangxiu = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'装修情况')]/../text()")
                        if zhuangxiu:
                            house_zhuangxiu = zhuangxiu[0].strip()
                        else:
                            house_zhuangxiu = "NA"
                        
                        jzjiegou = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'建筑结构')]/../text()")
                        if jzjiegou:
                            house_jzjiegou = jzjiegou[0].strip()
                        else:
                            house_jzjiegou = "NA"
                        
                        '''
                        house_gongnuan = (house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'供暖方式')]/../text()")[0]).replace(' ','')
                        '''
                        
                        tihu = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'梯户比例')]/../text()")
                        if tihu:
                            house_tihu = tihu[0].strip()
                        else:
                            house_tihu = "NA"
                        
                        
                        dianti = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'配备电梯')]/../text()")
                        if dianti:
                            house_dianti = dianti[0].strip()
                        else:
                            house_dianti = "NA"
                        
                        
                        jiaoyiquanshu = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'交易权属')]/../text()")
                        if jiaoyiquanshu:
                            house_jiaoyiquanshu = jiaoyiquanshu[0].strip()
                        else:
                            house_jiaoyiquanshu = "NA"

                        guapai =house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'挂牌时间')]/../text()")
                        if guapai:
                            house_guapai = guapai[0].strip()
                        else:
                            house_guapai = "NA"

                        
                        
                        yongtu = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'房屋用途')]/../text()")
                        if yongtu:
                            house_yongtu = yongtu[0].strip()
                        else:
                            house_yongtu = "NA"
                        
                        nianxian = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'房屋年限')]/../text()")
                        if nianxian:
                            house_nianxian = nianxian[0].strip()
                        else:
                            house_nianxian = "NA"
                        
                        suoshu = house_html.xpath("//div[@class='content']/ul/li/span[@class='label'][contains(text(),'房权所属')]/../text()")
                        if suoshu:
                            house_suoshu = suoshu[0].strip()
                        else:
                            house_suoshu = "NA"
                        
                        '''
                        house_dealTotalPrice = (house_html.xpath("//span[@class='dealTotalPrice']/i/text()")[0]).replace(' ','')
                        house_avg_price = (house_html.xpath("//div[@class='price']/b/text()")[0]).replace(' ','')
                        house_guapai_price = (house_html.xpath("//div[@class='msg']/span[contains(text(),'挂牌价格（万）')]/label/text()")[0]).replace(' ','')
                        house_chengjiaozhouqi = (house_html.xpath("//div[@class='msg']/span[contains(text(),'成交周期（天）')]/label/text()")[0]).replace(' ','')
                        house_tiaojia = (house_html.xpath("//div[@class='msg']/span[contains(text(),'调价（次）')]/label/text()")[0]).replace(' ','')
                        house_daikan = (house_html.xpath("//div[@class='msg']/span[contains(text(),'带看（次）')]/label/text()")[0]).replace(' ','')
                        '''
                        guanzhu = house_html.xpath("//div[@class='msg']/span[contains(text(),'关注（人）')]/label/text()")
                        if guanzhu:
                            house_guanzhu = guanzhu[0].strip()
                        else:
                            house_guanzhu = "NA"
                        
                        kanguo = house_html.xpath("//div[@class='msg']/span[contains(text(),'带看（次）')]/label/text()")
                        if kanguo:
                            house_kanguo = kanguo[0].strip()
                        else:
                            house_kanguo = "NA"
                            
                        '''
                        house_liulan = (house_html.xpath("//div[@class='msg']/span[contains(text(),'浏览（次）')]/label/text()")[0]).replace(' ','')
                        '''

                        '''
                        xiaoqu_id_list.append(xiaoqu_id)
                        xiaoqu_name_list.append(xiaoqu_name)
                        house_id_list.append(house_id)
                        '''
                        xiaoqu_name_list.append(xiaoqu_name)
                        xiaoqu_diqu_list.append(xiaoqu_diqu)
                        city_list.append(a_city)
                        xiaoqu_id_list.append(parts[4])
                        guapai_price_list.append(guapai_price)
                        chengjiao_zhouqi_list.append(chengjiao_zhouqi)
                        house_tiaojia_list.append(house_tiaojia)
                        house_liulan_list.append(house_liulan)
                        house_lianjia_list.append(house_lianjia)
                        
                        
                        house_huxing_list.append(house_huxing)
                        house_louceng_list.append(house_louceng)
                        house_mianji_list.append(house_mianji)
                        house_hxjiegou_list.append(house_hxjiegou)
                        house_taonei_list.append(house_taonei)
                        house_leixing_list.append(house_leixing)
                        house_chaoxiang_list.append(house_chaoxiang)
                        '''house_niandai_list.append(house_niandai)
                        '''
                        house_zhuangxiu_list.append(house_zhuangxiu)
                        house_jzjiegou_list.append(house_jzjiegou)
                        '''
                        house_gongnuan_list.append(house_gongnuan)
                        '''
                        house_tihu_list.append(house_tihu)
                        house_dianti_list.append(house_dianti)

                        house_jiaoyiquanshu_list.append(house_jiaoyiquanshu)

                        house_guapai_list.append(house_guapai)
                        
                        house_yongtu_list.append(house_yongtu)
                        house_nianxian_list.append(house_nianxian)
                        house_suoshu_list.append(house_suoshu)
                        '''
                        house_dealTotalPrice_list.append(house_dealTotalPrice)
                        house_avg_price_list.append(house_avg_price)
                        house_guapai_price_list.append(house_guapai_price)
                        house_chengjiaozhouqi_list.append(house_chengjiaozhouqi)
                        '''
                        '''
                        house_tiaojia_list.append(house_tiaojia)

                        house_daikan_list.append(house_daikan)
                        '''
                        house_guanzhu_list.append(house_guanzhu)
                        house_kanguo_list.append(house_kanguo)
                        '''
                        house_liulan_list.append(house_liulan)
                        '''

                    tmp_house_df = pd.DataFrame(map(list,zip(

                        city_list,
                        xiaoqu_name_list,
                        xiaoqu_id_list,
                        guapai_price_list,
                        chengjiao_zhouqi_list,
                        house_tiaojia_list,
                        house_deal_date_list,
                        house_liulan_list,
                        house_lianjia_list,
                        xiaoqu_diqu_list,
                        house_huxing_list,
                        house_louceng_list,
                        house_mianji_list,
                        house_hxjiegou_list,
                        house_taonei_list,
                        house_leixing_list,
                        house_chaoxiang_list,

                        house_zhuangxiu_list,
                        house_jzjiegou_list,

                        house_tihu_list,
                        house_dianti_list,
                        house_jiaoyiquanshu_list,
                        house_guapai_list,
                        
                        house_yongtu_list,
                        house_nianxian_list,
                        house_suoshu_list,

                        house_guanzhu_list,
                        house_kanguo_list,
                        house_url_list)),columns=df_cols)
                    #print(tmp_house_df.head())
                    #print("===============count===========:",count)
                    feather_filename = 'check_temp.feather'
                    feather.write_feather(tmp_house_df, feather_filename)   

                    result_house_df=pd.concat([result_house_df, tmp_house_df], axis=0)
                    
                soup = BeautifulSoup(r.text, 'html.parser')
                next_button = soup.find('button', {'id': 'next-button'})

                # 判断下一页按钮是否可点击，如果不可点击则退出循环
                if not next_button or 'disabled' in next_button.attrs:
                    break
                else :
                    print('该页面无剩余房源')
                    break
    result_house_df = result_house_df.drop_duplicates(subset=df_cols)
    feather_filename1 = 'check_result_chengjiao.feather'
    feather.write_feather(result_house_df, feather_filename1)   
    return result_house_df


In [3]:
all_xiaoqu=pd.read_feather('xiaoqulinks.feather')
all_xiaoqu_list=all_xiaoqu['Links'].tolist()
all_xiaoqu_list=list(set(all_xiaoqu_list))

In [4]:
import pickle

In [5]:
with open('xiaoqulist.pickle', 'rb') as f:
    xiaoqu_list_national = pickle.load(f)

In [6]:
xiaoqu_list_national=list(set(xiaoqu_list_national))

In [7]:
xiaoqu_new=list(set(xiaoqu_list_national)-set(all_xiaoqu_list))

In [8]:
new_tier_one=[]
for i in xiaoqu_new:
    parts1=i.split('/')
    if parts1[2] in ['cd.lianjia.com','cq.lianjia.com','hz.lianjia.com','wh.lianjia.com','su.lianjia.com','xa.lianjia.com','nj.lianjia.com','cs.lianjia.com','tj.lianjia.com','zz.lianjia.com','dg.lianjia.com','qd.lianjia.com','km.lianjia.com','nb.lianjia.com','hf.lianjia.com']:
        new_tier_one.append(i)
new_tier_one.sort()        
len(new_tier_one)

108754

In [9]:
new_tier_one_split=[new_tier_one[i:i+300] for i in range(0, len(new_tier_one), 300)]
len(new_tier_one_split)

363

In [11]:
chengjiao_df=pd.read_feather('lianjia_chengjiao.feather')
count=52
chengjiao_filename='lianjia_chengjiao.feather'
cookies='SECKEY_ABVK=wahsg0xUYIsSseGpV7ntH33DrJswDpw2Plw2QWZFmSYjcFqEJrXNjtLwCRr6wiqdKA7icH8b5SMHyKHlBRZnFQ%3D%3D; BMAP_SECKEY=S7bkVP4xQqoLxUlEaDjBNV2M9N9K3_nF2yKl0MP8NanOzd69AGPlv0yGgq-T-48I-Mv3LZNL6cFVoLlU6G6L_ln7X0vOSscJvrqmDDKmoEcZ1o55YBAYrr1_6RXfNqPfPlSGY6NpPUjl0Y5lhN0V5jrZ1Gq_66N8jLqXY0LagTmJ-UJkvVS2oLBIhYHR7IYsqb3UjRBrf4VDLzdVtUpkaw; lianjia_uuid=a1ecdfff-2722-472b-a0f8-3f0f09222ddb; crosSdkDT2019DeviceId=-5cav4c--pcg1jo-jlq4vkogud4ri8a-ffkgs9pa6; lfrc_=0e586a92-aa57-440f-993a-7db260d72049; _smt_uid=65040e16.5ebe813a; _ga=GA1.2.1912050430.1694764573; _jzqy=1.1695215405.1695215405.1.jzqsr=baidu.-; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2218a97d6fe713b4-0cfec44792da49-26031e51-1382400-18a97d6fe72394%22%2C%22%24device_id%22%3A%2218a97d6fe713b4-0cfec44792da49-26031e51-1382400-18a97d6fe72394%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; _ga_LRLL77SF11=GS1.2.1695915521.4.1.1695915545.0.0.0; _ga_GVYN2J1PCG=GS1.2.1695915521.4.1.1695915545.0.0.0; _jzqx=1.1694764564.1697625466.16.jzqsr=clogin%2Elianjia%2Ecom|jzqct=/.jzqsr=gz%2Elianjia%2Ecom|jzqct=/chengjiao/; _ga_654P0WDKYN=GS1.2.1697649303.6.0.1697649303.0.0.0; _ga_NKBFZ7NGRV=GS1.2.1697823269.1.0.1697823269.0.0.0; _ga_XLL3Z3LPTW=GS1.2.1697823269.1.0.1697823269.0.0.0; _ga_PV625F3L95=GS1.2.1697823278.1.0.1697823278.0.0.0; _ga_1W6P4PWXJV=GS1.2.1697823287.1.0.1697823287.0.0.0; _ga_W9S66SNGYB=GS1.2.1697823287.1.0.1697823287.0.0.0; _ga_BGW2B8P0NN=GS1.2.1697823295.1.1.1697823320.0.0.0; _ga_WLZSQZX7DE=GS1.2.1697823347.2.0.1697823347.0.0.0; _ga_TJZVFLS7KV=GS1.2.1697823347.2.0.1697823347.0.0.0; _ga_NKCLMZHBXY=GS1.2.1697823372.1.0.1697823372.0.0.0; _ga_WGKDF6B591=GS1.2.1697823394.1.0.1697823394.0.0.0; _ga_E91JCCJY3Z=GS1.2.1697823409.1.0.1697823409.0.0.0; _ga_MFYNHLJT0H=GS1.2.1697823409.1.0.1697823409.0.0.0; _ga_4JBJY7Y7MX=GS1.2.1697823418.1.0.1697823418.0.0.0; _ga_B3G62E46BE=GS1.2.1697823444.1.0.1697823444.0.0.0; _ga_049GGDBYWQ=GS1.2.1697823444.1.0.1697823444.0.0.0; _ga_8EKBN6G64V=GS1.2.1697823464.1.0.1697823464.0.0.0; _ga_EYZV9X59TQ=GS1.2.1697823470.1.0.1697823470.0.0.0; _ga_DX18CJBZRT=GS1.2.1697823470.1.0.1697823470.0.0.0; _ga_C4R21H79WC=GS1.2.1697901392.4.1.1697902049.0.0.0; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1697823773,1697900721,1698075221,1698159102; _ga_KJTRWRHDL1=GS1.2.1698159107.14.1.1698159125.0.0.0; _ga_QJN1VP0CMS=GS1.2.1698159107.14.1.1698159125.0.0.0; select_city=110000; lianjia_ssid=4d8edfd1-7dc8-43ec-9436-41dd6211cecc; _qzjc=1; _jzqa=1.3340631992986723000.1694764564.1698159097.1698250695.41; _jzqc=1; _jzqckmp=1; login_ucid=2000000367596316; lianjia_token=2.00120dc6887e6db87803a0efb99f2cdba7; lianjia_token_secure=2.00120dc6887e6db87803a0efb99f2cdba7; security_ticket=S6g6MIOieGW4MPDX8D2MUf372Qoes4/KOn7tL0sqaYFcjkvtstLUBLKr+IjLVNcGwGrul97S7rOtI/68uYRjMQZ4nUt1yKJtabYA7m3L2s3ndX0covDXhH3r+m8lvgjOtFauNO0ypHI/mdZMZyUnixLP/+LK05Yr472QjA1Ps30=; _qzja=1.471993342.1695141463227.1698159096813.1698250694725.1698250701693.1698250755436.0.0.0.60.16; _qzjb=1.1698250694724.3.0.0.0; _qzjto=3.1.0; _jzqb=1.3.10.1698250695.1; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiYzg0ZTc3OTdiNTI4M2I0ZGQwZmRlNzUzZTE4MWZiYTNhNTdkZjA0ZDI2MDc3MDYxNGFiNjVmNDZjYjBlY2YyYjZlMmY5YTNiNzMwZmQyM2ZjMTI3NTdmNTJjNTNjZWI4NWMxYTg1ZDU1NDMwNGQ1MjU3MWE2ZDcyYTk0ZWI4YWI3MWE0ZDM4ZGRiZGFmNTM1NzdkNzNlMWIyMTc1OTVmOGFmOTg1MWU5YWFkODg5YWU2MTlkYzFjOWJhZmM4YWUxYzdhN2ZjY2JhYWI1NGU3YWI0MTFkY2JiZDQ2OTRjYTY1MDFiNTE2YTUwODY0NjNkZDEwMGM3NDFlZjJlNDhlNlwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCIyOGUyY2EyZVwifSIsInIiOiJodHRwczovL2JqLmxpYW5qaWEuY29tL2NoZW5namlhby8iLCJvcyI6IndlYiIsInYiOiIwLjEifQ=='
for xiaoqu in new_tier_one_split[51:]:
    print(count)
    temp=get_xiaoqu_chengjiao(xiaoqu,cookies)
    chengjiao_df=pd.concat([chengjiao_df, temp], axis=0)
    feather.write_feather(chengjiao_df, chengjiao_filename)
    count+=1
    

52


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [44:00<00:00,  8.80s/it]


53


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [54:29<00:00, 10.90s/it]


54


...

72


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [12:06<00:00,  2.42s/it]


73


 91%|████████████████████████████████████████████████████████████████████████▊       | 273/300 [11:30<01:08,  2.53s/it]


KeyboardInterrupt: 

In [12]:
pd.read_feather('lianjia_chengjiao.feather')

,城市,小区名称,小区id,挂牌价格（万）,成交周期（天）,调价（次）,成交日期,浏览（次）,链家编号,所在区域,...,梯户比例,配备电梯,交易权属,挂牌时间,房屋用途,房屋年限,产权所属,关注次数,看房次数,房屋链接
0,sz,太古城北,2411049531541,2200,67,2,2020.08.15,4744,105104554622,(南山区深圳湾)中心路(深圳湾段)2233号,...,两梯四户,有,商品房,2020-06-09,普通住宅,满五年,非共有,36,NA,https://sz.lianjia.com/chengjiao/105104554622....
1,sz,太古城北,2411049531541,2150,459,3,2020.07.18,3736,105102259846,(南山区深圳湾)中心路(深圳湾段)2233号,...,两梯四户,有,商品房,2019-04-17,普通住宅,满五年,非共有,80,NA,https://sz.lianjia.com/chengjiao/105102259846....
2,sz,太古城北,2411049531541,1100,374,1,2019.11.15,438,105101728094,(南山区深圳湾)中心路(深圳湾段)2233号,...,三梯八户,有,商品房,2018-11-07,普通住宅,暂无数据,非共有,6,NA,https://sz.lianjia.com/chengjiao/105101728094....
3,sz,太古城北,2411049531541,1850,334,3,2019.03.27,427,105101338859,(南山区深圳湾)中心路(深圳湾段)2233号,...,两梯四户,有,商品房,2018-04-27,普通住宅,暂无数据,共有,68,NA,https://sz.lianjia.com/chengjiao/105101338859....
4,sz,太古城北,2411049531541,900,17,1,2019.03.27,209,105101081455,(南山区深圳湾)中心路(深圳湾段)2233号,...,三梯八户,有,商品房,2017-11-03,普通住宅,暂无数据,非共有,0,NA,https://sz.lianjia.com/chengjiao/105101081455....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,cd,博雅新城E区,3011056749384,95,128,0,2017.09.23,218,106100164361,(新都大丰)晓初滨河路一巷222号,...,两梯四户,有,商品房,2017-03-03,普通住宅,暂无数据,共有,121,NA,https://cd.lianjia.com/chengjiao/106100164361....
25,cd,博雅新城E区,3011056749384,60,81,0,2017.07.08,220,106100233421,(新都大丰)晓初滨河路一巷222号,...,两梯三户,有,商品房,2017-03-30,普通住宅,暂无数据,非共有,41,NA,https://cd.lianjia.com/chengjiao/106100233421....
26,cd,博雅新城E区,3011056749384,78,14,0,2017.06.18,212,106100389030,(新都大丰)晓初滨河路一巷222号,...,两梯三户,有,商品房,2017-06-03,普通住宅,暂无数据,非共有,2,NA,https://cd.lianjia.com/chengjiao/106100389030....
27,cd,博雅新城E区,3011056749384,62,7,0,2017.06.17,211,106100355833,(新都大丰)晓初滨河路一巷222号,...,两梯三户,有,商品房,2017-05-20,普通住宅,暂无数据,共有,2,NA,https://cd.lianjia.com/chengjiao/106100355833....


In [12]:
a=shanghai_new_split[17]
a[292]

'https://sh.lianjia.com/xiaoqu/5011000012822/'

In [15]:
a[293]

'https://sh.lianjia.com/xiaoqu/5011000006079/'